# Atlas

In this notebook, you will learn how to use `atlas` to conduct data-driven autonomous experimentation.

![Atlas banner logo](https://github.com/aspuru-guzik-group/atlas/blob/main/static/atlas_logo.png)

Notes:
- when executing the first cell, you will receive a warning ("This notebook was not authored by Google"). Please select "Run Anyway" to be able to run the cells of the notebook.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os, sys

In [ ]:
# install olympus first
!git clone -b olympus-atlas --single-branch https://github.com/aspuru-guzik-group/olympus.git
%cd 'olympus'
!pip install -e .
%cd ../

# install atlas from source
!git clone https://github.com/aspuru-guzik-group/atlas.git
%cd 'atlas'
!pip install -e .
!pip install -r requirements.txt
%cd ../
sys.path.append('atlas/src/')

In [1]:
# import numerical programming and data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from olympus.campaigns import ParameterSpace
from olympus.surfaces import Surface
from olympus.objects import (
    ParameterContinuous,
    ParameterDiscrete,
    ParameterCategorical,
    ParameterVector,
)


## Proof-of-concept optimization

In this example, we will see how to perform simple optimizations using `atlas`. In the first example, we will see optimization of an a chemical reaction with a
fully continuous parameter space. Specifically, the reaction is the biocatalytic oxidation of benzyl alcohol by a copper radical oxidase (AlkOx). The effects of enzyme loading, cocatalyst loading, and pH balance on conversion are studied. 

`atlas` interfaces with the `olympus` Python package, which provides an easy way to interact with optimization datasets. Learn more about `olympus` from its [documentation](https://aspuru-guzik-group.github.io/olympus/) and [GitHub repo](https://github.com/aspuru-guzik-group/olympus). First, lets load in the `alkox` dataset from `olympus` and visualize it. The `Dataset` object of `olympus` wraps a pandas DataFrame in its `data` attribute. This dataset consists of 4 continuous parameters (`catalase`, `peroxidase`, `alcohol_oxidase` and `ph`) and 1 objective, which is to be maximized. [cite olympus paper]

In [2]:
from olympus.datasets import Dataset

dset = Dataset(kind='alkox')
dset.data.head()

,catalase,peroxidase,alcohol_oxidase,ph,conversion
0,0.05,0.5,2.0,6.0,5.932566
1,0.05,0.5,2.0,6.0,5.932566
2,0.05,0.5,2.0,7.0,2.173315
3,0.05,0.5,2.0,7.0,2.173315
4,0.05,0.5,2.0,8.0,1.056175


`olympus` also provides `Emulators`, which are probabilistic nerual networks trained to virtually reproduce experimental measurements. This is convenient for debugging or benchmarking optimizers on realistic tasks derived from chemistry and materials science research. By passing parameters to the emulator, you can get back virtual measurements. Lets try this for the `alkox` emulator. 

In [3]:
from olympus.emulators import Emulator 

emulator = Emulator(dataset='alkox', model='BayesNeuralNet')

params = [np.random.uniform(p.low, p.high, size=None) for p in emulator.param_space]

measurement, _, __ = emulator.run(params, return_paramvector=True)
print('params : ', params)
print('measurement : ', measurement)

[INFO] Loading emulator using a BayesNeuralNet model for the dataset alkox...


/Users/rileyhickman/mambaforge/envs/atlas_aag/lib/python3.10/site-packages/tensorflow_probability/python/layers/util.py:98: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
/Users/rileyhickman/mambaforge/envs/atlas_aag/lib/python3.10/site-packages/tensorflow_probability/python/layers/util.py:108: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


params :  [0.6869624280556538, 2.499922095397715, 7.574343922858796, 7.786808550872344]
measurement :  [ParamVector(conversion = 7.246413582692234)]


Finally, lets see how to conduct an optimization using `atlas`. We can instantiate the `GPPlanner` object from `atlas`. This will serve as our default GP-based Bayesian optimization strategy, and will automaticallty adapt to the parameter space and objective values of our task. For instance, `alkox` is a "fully continuous" parameter problem, therefore, `atlas` will use a Matern5/2 kernel. By default, it will use the expected improvement acquisition function, and a gradient based strategy to optimize the acquisition function. Lets instantiate the planner. After that, we can sequentially ask the `planner` for parameters, the `emulator` for measurements, and store this information neatly in a `Campaign` instance from `olympus`. 

In [5]:
from atlas.planners.gp.planner import GPPlanner
from olympus.campaigns import Campaign

# instantiate atlas planner
planner = GPPlanner(
      goal='maximize',
      init_design_strategy='lhs',
      acquisition_optimizer_kind='gradient',
      num_init_design=5,
      batch_size=1,
)
planner.set_param_space(emulator.param_space)

# instantiate campaign
campaign = Campaign()
campaign.set_param_space(emulator.param_space)

# commence the optimization experiment
BUDGET = 10
iter_ = 0
while len(campaign.observations.get_values()) < BUDGET:

    # ask atlas for parameters to measure next
    samples = planner.recommend(campaign.observations)

    # evaluate samples using the emulator and add observation to 
    # the Olympus campaign object
    for sample in samples:
        measurement = emulator.run(sample, return_paramvector=True)
        print(f'\nITER : {iter_+1}\nSAMPLE : {sample}\nMEASUREMENT : {measurement[0]}\n')
        campaign.add_observation(sample, measurement[0])
        
        iter_+=1



───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                                                                                   
                                                 Welcome to ATLAS!                                                 

                                                Made with 💕 in 🇨🇦                                                 
                                                                                                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

───────────────────────────── Initial design phase ─────────────────────────────

...

───────────────────── Training regression surrogate model ──────────────────────

/Users/rileyhickman/mambaforge/envs/atlas_aag/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


ATLAS [INFO] Regression surrogate GP trained in 0.038 sec

────────────────────── Acquisition function optimization ───────────────────────


ITER : 10
SAMPLE : ParamVector(catalase = 0.7260903796143366, peroxidase = 9.999999640119826, alcohol_oxidase = 7.270178673569694, ph = 6.627723781215826)
MEASUREMENT : [ParamVector(conversion = 19.765606228577358)]



Next, we will see how to use `atlas` to optimize a fully categorical problem with descriptors. Categorical parameters are extremely important in the experimental sciences, and are characterized by a lack of inate order between the variable options. In this example, we will use the `perovskites` dataset from `olympus`, which reports bandgap values (to be minimized) for 192 unique hybrid organic-inorganic perovskite materials. [add citation]

For categorical variables, we can also featurize the options with vectors of descriptors to induce an ordering between them, and potentially increase the optimization rate. For example, `olympus`' `perovskites` dataset ships with geometric and electronic descriptors of the 3 perovskite components (`organic`, `cation` and `anion`).

In [7]:
dset = Dataset(kind='perovskites')
dset.data.head()

,organic,cation,anion,hse_gap
0,ethylammonium,Ge,F,5.3704
1,ethylammonium,Ge,Cl,3.1393
2,ethylammonium,Ge,Br,2.7138
3,ethylammonium,Ge,I,2.2338
4,ethylammonium,Sn,F,3.9789


In [8]:
dset.descriptors.head()

,param,option,name,value
0,organic,ethylammonium,homo,-0.4601
1,organic,ethylammonium,lumo,-0.22398
2,organic,ethylammonium,dipole,1.3965
3,organic,ethylammonium,atomization,-1.84142
4,organic,ethylammonium,r_gyr,1.261565


We can instantiate the `GPPlanner` in the same way as in the continuious case, and it will automaticallty adapt to the fully categorical problem. The `use_descriptors` argument specifies whether or not to use descriptors of the categorical variable options. If we have no descritpors (one-hot-encoded representation), `atlas` will use a Categorical kernel based on Hamming distances. If we do have descriptors, `atlas` will use the Matern5/2 kernel. The acquisition optimization also differs in the fully categorical case from the continuous case. Try flipping the `use_descriptors` argument to `True`. Note that we've also pivoted the `init_design_strategy` to `random` from `lhs` (Latin Hypercube Sampling), which will not work with categorical parameter spaces.

In [9]:
# instantiate atlas planner
planner = GPPlanner(
      goal='minimize',
      init_design_strategy='random', 
      num_init_design=5,
      batch_size=1,
      use_descriptors=True,
)
planner.set_param_space(dset.param_space)

# instantiate campaign
campaign = Campaign()
campaign.set_param_space(dset.param_space)

# commence the optimization experiment
BUDGET = 10
iter_ = 0
while len(campaign.observations.get_values()) < BUDGET:

    # ask atlas for parameters to measure next
    samples = planner.recommend(campaign.observations)

    # evaluate samples using the emulator and add observation to 
    # the Olympus campaign object
    for sample in samples:
        measurement = dset.run(sample, return_paramvector=True)
        print(f'\nITER : {iter_+1}\nSAMPLE : {sample}\nMEASUREMENT : {measurement[0]}\n')
        campaign.add_observation(sample, measurement[0])
        
        iter_+=1

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                                                                                   
                                                 Welcome to ATLAS!                                                 

                                                Made with 💕 in 🇨🇦                                                 
                                                                                                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

───────────────────────────── Initial design phase ─────────────────────────────

...

───────────────────── Training regression surrogate model ──────────────────────

/Users/rileyhickman/mambaforge/envs/atlas_aag/lib/python3.10/site-packages/botorch/models/utils/assorted.py:173: InputDataWarning: Input data is not contained to the unit cube. Please consider min-max scaling the input data.
  warnings.warn(msg, InputDataWarning)
/Users/rileyhickman/mambaforge/envs/atlas_aag/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


ATLAS [INFO] Regression surrogate GP trained in 0.019 sec

────────────────────── Acquisition function optimization ───────────────────────


ITER : 10
SAMPLE : ParamVector(organic = ethylammonium, cation = Sn, anion = F)
MEASUREMENT : ParamVector(hse_gap = 4.014873442419595)



## Optimization of mixed-parameter spaces

Mixed parameter spaces are defined as those with heterogenous parameter types. These parameter spaces are extremely important in chemistry and materials science. Researchers often seek to choose categorical or discrete options (e.g. catalyst ligand for a chemical reaction) while simultaneously tuning continious parameters (e.g. reaction temperature, reaction time, substrate concentrations). 

As an illustrative example, lets consider the `suzuki_iii` dataset from `olympus`, which reports the yeild and catalyst turnover number for flow-based Suzuki-Miyaura cross-coupling reactions with varying substrates. There are three continous parameters (temperature, residence time, and catalyst loading) and one categorical parameter (Pd catalyst ligand). The objective is to simultaneously maximize both the yield and catalyst turnover number. 



## Optimization with a priori known constraints

_A priori_ known parameter constraints are a pervasive topic in experimental science optimization campaigns. Although the value of the constraint function $c(\mathbf{x})$ is known beforehand by the researcher, the constraints may be interdependent, non-linear, and result in non-compact optimization domains. For example, when optimizing the yield of a chemical reaction, one might want the temperature, $T$, to be varied in the interval $10 < T < 100^{\circ}\text{C}$ for experiments using water as the solvent, but in the interval $10 < T < 66^{\circ}\text{C}$ for experiments using THF. 

`atlas` provides a simple, flexible interface to deal with known constraints on the parameter space. To the constructor of any of its planners, users may provide a list of Python callables which, for a single set of input parameters, return a boolean: `True` if the set of parameters is feasible, and `False` if not. Lets consider a simple example using the 2d `Dejong` surface from `olympus`.

In [10]:
from olympus.surfaces import Surface

# intialize Dejong surface (2d parameter space by default)
surface = Surface(kind='Dejong')

# define the constraint function
def known_constraint(params):
    # params is a array-like object representing one parameter setting
    y = (params[0]-0.5)**2 + (params[1]-0.5)**2
    if np.abs(params[0]-params[1]) < 0.1:
        return False
    if 0.05 < y < 0.15:
        return False
    else:
        return True

Next, we initialize the planner, and pass a list of the known constraint callables for the `known_constraints` argument (here we only have one callable, but `atlas` supports evaluation of an arbitrary number). 

In [11]:
# initialize the planner with kwown constraints
planner = GPPlanner(
        goal="minimize",
        init_design_strategy='random',
        num_init_design=5,
        batch_size=1,
        acquisition_optimizer_kind='pymoo',
        known_constraints=[known_constraint],
    )
planner.set_param_space(surface.param_space)

# initialize campaign
campaign = Campaign()
campaign.set_param_space(surface.param_space)

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                                                                                   
                                                 Welcome to ATLAS!                                                 

                                                Made with 💕 in 🇨🇦                                                 
                                                                                                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

───────────────────────────── Initial design phase ─────────────────────────────

In [12]:
# commence experiment
BUDGET = 20
iter_ = 0
while len(campaign.observations.get_values()) < BUDGET:

    samples = planner.recommend(campaign.observations)
    for sample in samples:
        measurement = surface.run(sample)[0][0]

        print(f'\nSAMPLE : {sample}\nMEASUREMENT : {measurement}\n')
        campaign.add_observation(sample, measurement)

        iter_ += 1

ATLAS [INFO] Generating 1 initial design points (batch 1/5)


SAMPLE : ParamVector(param_0 = 0.6555782987321565, param_1 = 0.3791476384108612)
MEASUREMENT : 2.3466390126198697



ATLAS [INFO] Generating 1 initial design points (batch 2/5)


SAMPLE : ParamVector(param_0 = 0.022081075480914647, param_1 = 0.2323229486688222)
MEASUREMENT : 3.822219583295823



ATLAS [INFO] Generating 1 initial design points (batch 3/5)

...

ATLAS [INFO] Regression surrogate GP trained in 0.079 sec

────────────────────── Acquisition function optimization ───────────────────────

ATLAS [INFO] Optimizing acquisition function with pymoo GA for 0 fixed parameters...

ATLAS [INFO] Completed in 0.971 sec


SAMPLE : ParamVector(param_0 = 0.47359115505492594, param_1 = 0.5736902362829905)
MEASUREMENT : 1.3723255417253477



## Optimization with a priori unknown constraints

The difference between _a priori_ known and unknown parameter constraints is that the latter involves a constraint function $c(\mathbf{x})$ that is not known beforehand by the researcher, and must be resolved by sequential measurement. `atlas` handles such optimization problems by training two surrogate models. The first is the usual regression GP that learns the objective function $f(\mathbf{x})$ and informs the acquisition function $\alpha(\mathbf{x})$. The second surrogate uses a variational GP to model the binary constraint or feasibility function, $c(\mathbf{x})$. Essentially, the classification surrogate seeks to model the posterior $P(feasible | \mathbf{x})$. The acquisition function $\alpha(\mathbf{x})$ and feasibility posterior can then be combined in various ways to produce a _feasibility-aware acquisition function_, $\alpha_c(\mathbf{x})$. `atlas` supports several such acquisition functions, for more information, please see the [publication]() or [documentation](). 

In this example, we will use the same surface and constraint funciton as in the known constraints example, but assume the form of $c(\mathbf{x})$ is _a priori_ unknown. We will used the so-called _feasibility weighted_ acquisition function (FWA), which has the following form. 

$$ \alpha_c(\mathbf{x}) = \alpha(\mathbf{x}) P\left(feasible|\mathbf{x}\right)$$

In [13]:
# intialize Dejong surface (2d parameter space by default)
surface = Surface(kind='Dejong')

# define the constraint function
def unknown_constraint(params):
    print(params)
    # params is a array-like object representing one parameter setting
    y = (params[0]-0.5)**2 + (params[1]-0.5)**2
    if np.abs(params[0]-params[1]) < 0.1:
        return False
    if 0.05 < y < 0.15:
        return False
    else:
        return True

In [14]:
# initialize the planner indicating the
# feasibility acquisition strategy, FWA
planner = GPPlanner(
        goal="minimize",
        feas_strategy='fwa',
        init_design_strategy='random',
        num_init_design=5,
        batch_size=1,
        acquisition_optimizer_kind='pymoo',
    )
planner.set_param_space(surface.param_space)

# initialize campaign
campaign = Campaign()
campaign.set_param_space(surface.param_space)

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                                                                                   
                                                 Welcome to ATLAS!                                                 

                                                Made with 💕 in 🇨🇦                                                 
                                                                                                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

───────────────────────────── Initial design phase ─────────────────────────────

In [15]:
# commence experiment
BUDGET = 20
iter_ = 0
while len(campaign.observations.get_values()) < BUDGET:

    samples = planner.recommend(campaign.observations)
    for sample in samples:
        
        # evaluate constrained surface, return Nan if infeasible point
        if unknown_constraint(sample.to_array()):
            measurement = measurement = surface.run(sample)[0][0]
        else:
            measurement = np.nan

        print(f'\nITER : {iter_+1}\nSAMPLE : {sample}\nMEASUREMENT : {measurement}\n')
        campaign.add_observation(sample, measurement)

        iter_+=1

ATLAS [INFO] Generating 1 initial design points (batch 1/5)

[0.0575479  0.42158718]

ITER : 1
SAMPLE : ParamVector(param_0 = 0.05754789949823069, param_1 = 0.4215871769100561)
MEASUREMENT : 2.9889646936596845



ATLAS [INFO] Generating 1 initial design points (batch 2/5)

[0.76739094 0.3145235 ]

ITER : 2
SAMPLE : ParamVector(param_0 = 0.7673909382694225, param_1 = 0.31452349712352634)
MEASUREMENT : nan



ATLAS [INFO] Generating 1 initial design points (batch 3/5)

...

Output()

ATLAS [INFO]  Classification surrogate VGP trained in 5.366 sec (1999 epochs)    Loss : 25.021

Output()

ATLAS [INFO]  Classification surrogate VGP trained in 5.286 sec (1999 epochs)    Loss : 1.254

KeyboardInterrupt: 

## Multi-objective optimization

Optimization problems in the experimental sciences often feature multiple, potentially competing objectives which must be optimized simultaneously. `atlas` allows for multi-objective optimization via _achivement_ _scalarizing_ _functions_ (ASFs) implemented in the `olympus` package.


As an example, lets consider the `redoxmers` dataset from `olympus`, which concerns the design of redox-active materials for flow batteries [cite redoxmers paper]. This dataset has a fully categorical parameter space and has 3 objectives, all of which are to be minimized. The dataset reports maximum absorption wavelengths, reduction potentials against a Li/Li+ reference electrode, and solvation free energies computed using DFT for a dataset of 1408 benzothiadiazole derivatives. The molecules in this dataset are screened as candidates for self-reporting redox-active materials for non-aqeuous redox flow batteries. We provide simple physicochemical descriptors for each of the substituents. [cite paper]

`olympus` provides an interface with several ASFs, including Chimera, Hypervolume indicator, Chebyshev, and WeightedSum. In this example, we will be using the Hypervolume indicator

In [16]:
# load dataset
dset = Dataset(kind='redoxmers')
dset.data.head()

,r1_label,r3_label,r4_label,r5_label,abs_lam_diff,ered,gsol
0,R1_0,R3_0,R4_0,R5_0,39.96,1.684123,-0.681801
1,R1_0,R3_0,R4_0,R5_1,63.92,1.963624,-0.711542
2,R1_0,R3_0,R4_0,R5_2,51.76,2.044655,-0.8874
3,R1_0,R3_0,R4_0,R5_3,36.93,1.731604,-0.710235
4,R1_0,R3_0,R4_0,R5_4,53.79,1.844226,-0.748112


In [17]:
params = ParameterVector().from_dict({
    'r1_label': 'R1_0',
    'r3_label': 'R3_0',
    'r4_label': 'R4_0',
    'r5_label': 'R5_0',
})

measurement = dset.run(params, return_paramvector=True)
print(measurement)
print(dset.value_space)

[ParamVector(abs_lam_diff = 40.05554315879696, ered = 1.6751627567041005, gsol = -0.629535661453569)]
Continuous (name='abs_lam_diff', low=0.0, high=1.0, is_periodic=False)
Continuous (name='ered', low=0.0, high=1.0, is_periodic=False)
Continuous (name='gsol', low=0.0, high=1.0, is_periodic=False)


In [18]:
planner = GPPlanner(
        goal='minimize',
        num_init_design=5,
        batch_size=1,
        acquisition_optimizer_kind='gradient',
        is_moo=True, 
        scalarizer_kind='Hypervolume',
        value_space=dset.value_space,
        goals=['min', 'min', 'min']
    )
planner.set_param_space(dset.param_space)

# initialize campaign
campaign = Campaign()
campaign.set_param_space(dset.param_space)
campaign.set_value_space(dset.value_space)

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                                                                                   
                                                 Welcome to ATLAS!                                                 

                                                Made with 💕 in 🇨🇦                                                 
                                                                                                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

───────────────────────────── Initial design phase ─────────────────────────────

In [19]:
# commence experiment
BUDGET = 20
iter_ = 0
while len(campaign.observations.get_values()) < BUDGET:

    samples = planner.recommend(campaign.observations)
    for sample in samples:
        
        measurement = measurement = dset.run(sample, return_paramvector=True)

        print(f'\nITER : {iter_+1}\nSAMPLE : {sample}\nMEASUREMENT : {measurement}\n')
        campaign.add_observation(sample, measurement)

        iter_+=1

ATLAS [INFO] Generating 1 initial design points (batch 1/5)


ITER : 1
SAMPLE : ParamVector(r1_label = R1_0, r3_label = R3_1, r4_label = R4_7, r5_label = R5_3)
MEASUREMENT : [ParamVector(abs_lam_diff = 24.293259379608923, ered = 1.945570696581578, gsol = -0.7499211103759299)]



ATLAS [INFO] Generating 1 initial design points (batch 2/5)


ITER : 2
SAMPLE : ParamVector(r1_label = R1_0, r3_label = R3_5, r4_label = R4_4, r5_label = R5_5)
MEASUREMENT : [ParamVector(abs_lam_diff = 13.222998818675913, ered = 2.1694638641232915, gsol = -0.7869681307142566)]



ATLAS [INFO] Generating 1 initial design points (batch 3/5)

...


ITER : 19
SAMPLE : ParamVector(r1_label = R1_0, r3_label = R3_0, r4_label = R4_1, r5_label = R5_1)
MEASUREMENT : [ParamVector(abs_lam_diff = 54.69966140174285, ered = 2.3787385134043166, gsol = -0.8293243918269284)]



───────────────────── Training regression surrogate model ──────────────────────

ATLAS [INFO] Regression surrogate GP trained in 0.024 sec

────────────────────── Acquisition function optimization ───────────────────────


ITER : 20
SAMPLE : ParamVector(r1_label = R1_0, r3_label = R3_0, r4_label = R4_1, r5_label = R5_2)
MEASUREMENT : [ParamVector(abs_lam_diff = 40.27237259092415, ered = 2.42979897711986, gsol = -0.7061758514859319)]



## Robust optimization with Golem

`Golem` is an algorithm for robust optimization, and helps identify optimal solutions that are robust to uncertainty on input parameters, ensuring the reproducibility of experimental protocols and processes. `golem` can be used in conjunction with any optimization algorithm or design of experiment strategy. For more information about `golem`, please refer to the [publication](https://pubs.rsc.org/en/content/articlelanding/2021/sc/d1sc01545a), [GitHub repo](https://github.com/aspuru-guzik-group/golem), and [documentation](https://aspuru-guzik-group.github.io/golem/). 

`atlas` supports use of `golem` in tandem with any of its planners. The following provides a simple 3d continuous parameter optimization. 


In [20]:
# define the toy surface and parameter space
def surface(x):
    return np.sin(8 * x[0]) - 2 * np.cos(6 * x[1]) + np.exp(-2.0 * x[2])

param_space = ParameterSpace()
param_0 = ParameterContinuous(name="param_0", low=0.0, high=1.0)
param_1 = ParameterContinuous(name="param_1", low=0.0, high=1.0)
param_2 = ParameterContinuous(name="param_2", low=0.0, high=1.0)
param_space.add(param_0)
param_space.add(param_1)
param_space.add(param_2)


To utilize `golem`, users must provide an argument to the planner constructor called `golem_config`. This argument must be a dictionary, where the keys are parameter names, and the values specify the types and parameterization of the uncertainty distributions for that input parameter. The values of these dictionaries can either be dictionaries themselves, or instances of `golem.BaseDist` objects. 

In this example, the first two parameters will have `golem.Normal` uncertainties (with stdev of 0.2 and 0.3, respectively). The third parameter is ommitted from the `golem_config` argument on purpose. Any parameter which is present in the parameter space and is not included in `golem_config` will be assigned a `golem.Delta` distribution, indicating no uncertainty. 


In [21]:
# intialize atlas GP planner
planner = GPPlanner(
        goal="minimize",
        init_design_strategy='lhs',
        num_init_design=5,
        batch_size=1,
        golem_config = {
            'param_0': {'dist_type':'Normal', 'dist_params':{'std':0.2}},
            'param_1': {'dist_type':'Normal', 'dist_params':{'std':0.3}},
        },
    )

planner.set_param_space(param_space)

# intialize campaign
campaign = Campaign()
campaign.set_param_space(param_space)

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

                                                                                                                   
                                                 Welcome to ATLAS!                                                 

                                                Made with 💕 in 🇨🇦                                                 
                                                                                                                   

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

───────────────────────────── Initial design phase ─────────────────────────────

ATLAS [WARNING] No distribution requested for parameter param_2. Resorting to Delta distribution...

In [22]:
# commence optimization experiment
BUDGET = 10
iter_ = 0
while len(campaign.observations.get_values()) < BUDGET:

    samples = planner.recommend(campaign.observations)
    for sample in samples:
        sample_arr = sample.to_array()
        measurement = surface(sample_arr)
        print(f'\nITER : {iter_+1}\nSAMPLE : {sample}\nMEASUREMENT : {measurement}\n')
        campaign.add_observation(sample_arr, measurement)
        
        iter_+=1

ATLAS [INFO] Generating 1 initial design points (batch 1/5)


ITER : 1
SAMPLE : ParamVector(param_0 = 0.7955968767475703, param_1 = 0.5534345920651962, param_2 = 0.6375493023903006)
MEASUREMENT : 2.328941786393177



ATLAS [INFO] Generating 1 initial design points (batch 2/5)


ITER : 2
SAMPLE : ParamVector(param_0 = 0.31595853464237444, param_1 = 0.7639193549754782, param_2 = 0.8813261537274008)
MEASUREMENT : 1.0047016445934174



ATLAS [INFO] Generating 1 initial design points (batch 3/5)

...

───────────────────── Training regression surrogate model ──────────────────────

/Users/rileyhickman/mambaforge/envs/atlas_aag/lib/python3.10/site-packages/botorch/models/utils/assorted.py:201: InputDataWarning: Input data is not standardized. Please consider scaling the input to zero mean and unit variance.
  warnings.warn(msg, InputDataWarning)


ATLAS [INFO] Regression surrogate GP trained in 0.038 sec

────────────────────── Acquisition function optimization ───────────────────────


ITER : 10
SAMPLE : ParamVector(param_0 = 0.5567755259176129, param_1 = 0.1520338954107815, param_2 = 0.21383249465732768)
MEASUREMENT : -1.5388346106552189



## Optimization for a generalizable set of parameters

Often, researchers may want to find parameter sets that are _generalizable_.
For example, one might want to find a single set of chemical reaction conditions which give good yield across several different substrates. [cite MADNESS Science paper]

Consider an optimization problem with $d$ continuous reaction parameters, $\mathcal{X} \in \mathbb{R}^d$
(functional parameters), and a set of $n$ substrates $\mathcal{S} = { s_i }_{i=1}^n$ (non-functional
parameters). The goal of such an optimization is to maximize the objective function $f(\mathbf{x})$, which is
the average yield response across all substrate molecules,

$$ f_{\mathcal{C}} = \frac{1}{n} \sum_{i=1}^n f(\mathbf{x}, s_i)  . $$

For a minimization problem, the best performing parameters are

$$  \mathbf{x}^* = argmin_{\mathbf{x}\in \mathcal{X}, s_i \in \mathcal{C}} f_{\mathcal{C}}  .$$

`atlas` employs an approach which removes the need to measure the full $f_{\mathcal{C}}$ at each iteration. Consider a toy problem,
where $n=3$, and the following piecewise function is used for $f_{\mathcal{C}}$, and is to be minimized.

$$ f(\mathbf{x}, s) = \sin(x_1) + 12\cos(x_2) - 0.1x_3   \text{  if}  s = s_1$$

$$ f(\mathbf{x}, s) = 3\sin(x_1) + 0.01\cos(x_2) + x_3^2  \text{  if }  s = s_2$$

$$ f(\mathbf{x}, s) = 5\cos(x_1) + 0.01\cos(x_2) + 2x_3^3  \text{  if } s = s_3$$


The variable $s$ is a categorical parameter with 3 options. $f_{\mathcal{C}}$ has a minimum value of approximately
3.830719 at $\mathbf{x}^* = (0.0, 1.0, 0.0404)$. Given the appropriate `olympus` parameter space, one can instantiate the planner as follows.

In [ ]:
# define the surface, parameter space, and campaign

def surface(x, s):
    if s == '0':
        return  np.sin(x[0])+ 12*np.cos(x[1]) - 0.1*x[2]
    elif s == '1':
        return 3*np.sin(x[0])+ 0.01*np.cos(x[1]) + 1.*x[2]**2
    elif s == '2':
        return 5*np.cos(x[0])+ 0.01*np.cos(x[1]) + 2.*x[2]**3


# make parameter space
param_space = ParameterSpace()

# add general parameter (one-hot-encoded)
param_space.add(
    ParameterCategorical(
        name='s',
        options=[str(i) for i in range(3)],
        descriptors=[None for i in range(3)],      
    )
)
# add the three functional parameters
param_space.add(
    ParameterContinuous(name='x_1',low=0.,high=1.)
)
param_space.add(
    ParameterContinuous(name='x_2',low=0.,high=1.)
)
param_space.add(
    ParameterContinuous(name='x_3',low=0., high=1.)
)

campaign = Campaign()
campaign.set_param_space(param_space)

In [ ]:
# create planner
planner = GPPlanner(
    goal='minimize',
    init_design_strategy='random',
    num_init_design=5,
    batch_size=1,
    acquisition_type='general',
    acquisition_optimizer_kind='pymoo',
    general_parmeters=[0],
    
)
planner.set_param_space(param_space)


The `general_parameters` argument to the constructor takes a list of integers, which
represent the parameter space indices which are intended to be treated as _general_ or _non-functional_
parameters. The figure below shows the performance of `atlas` compared to random sampling on this toy
problem (10 repeats).

![alt text](https://github.com/rileyhickman/atlas/blob/main/static/synthetic_general_conditions_gradient.png)

In [ ]:
true_measurements = []


BUDGET = 10
iter_ = 0
while len(campaign.observations.get_values()) < BUDGET:
    
    samples = planner.recommend(campaign.observations)
    for sample in samples:
        # make the measurement for the recommended sample
        measurement = surface(
            [float(sample.x_1), float(sample.x_2), float(sample.x_3)],
            sample.s,
        )

        # evaluate the "true" objective function by averaging the functional parametrers
        # selected by the optimizer over all the non-functional parameter options
        all_measurements = []
        for s in param_space[0].options:
            all_measurements.append(
                surface(
                    [float(sample.x_1), float(sample.x_2), float(sample.x_3)],
                    s,
                )
            )
        true_measurements.append(np.mean(all_measurements))
        
        iter_+=1


    print(f'ITER : {iter_}\tSAMPLES : {samples}\t MEASUREMENT : {measurement}')
    campaign.add_observation(samples, measurement)
    
    


## Multi-fidelity optimization

Multi-fidelity Bayesian optimization targets problems where two or more “information sources”
are available to the researcher. Typically, the information sources generate measurements of the same property
at different levels of fidelity, precision, or accuracy, and are available at varying cost. For instance, a chemical property could be estimated using a crude but inexpensive simulation (low-fidelity) as a proxy for an accurate
but expensive experimental determination (high-fidelity). Atlas provides a `MultiFidelityPlanner` based on
the trace-aware knowledge gradient acquisition function which allows for the inclusion of an arbitrary
number of information sources with discrete fidelity levels.

For this illustrative example, we will use two fidelity levels, which are both 1d trigonometric functions. Lets define the objective functions.

In [ ]:
def surface(params):
    if params['s'] == 0.5:
        # low fidelity
        return np.sin(3.*params['x0']) - np.exp(-2.*params['x1'])
    elif params['s'] == 1.0: 
    # target fidelity
        return np.sin(2.*params['x0']) - np.exp(-1.*params['x1'])

We'll build the `olympus` parameter space for this problem. Note that we define an extra parameter called `s`, which is the _fidelity parameter_. This indicates the fidelity level for each parameter set. Conventionally, this must be a `ParameterDiscrete` instance with options between 0 and 1. $s=1.0$ is reserved for the target fidelity (or highest fidelity), with lower fidelities taking values $< 1.0$. We will use a value of 0.5 for the low fidelity measurements.

We'll next import the `MultiFidelityPlanner` from `atlas` and instantiate it. We need to pass a few additional arguments to the constructor. `feas_params` is an integer which is the parameter space index containing the fidelity parameter. In our case this is 2. `fidelities` is a list of floats representing the discrete fidelity levels. This is the same list as our list of options in the parameter space definition. 

In [ ]:
from atlas.planners.multi_fidelity.planner import MultiFidelityPlanner

param_space = ParameterSpace()

# two continuous parameters and one fidelity param
param_space.add(ParameterContinuous(name='x0', low=0., high=1.))
param_space.add(ParameterContinuous(name='x1', low=0., high=1.))
param_space.add(ParameterDiscrete(name='s', options=[0.5, 1.0]))

planner = MultiFidelityPlanner(
    goal='minimize', 
    init_design_strategy='random',
    num_init_design=5, 
    batch_size=1, 
    acquisition_optimizer_kind='pymoo',
    fidelity_params=2,
    fidelities=[0.5, 1.0],
)

planner.set_param_space(param_space)

campaign = Campaign()
campaign.set_param_space(param_space)

BUDGET = 20

iter_ = 0 
while len(campaign.observations.get_values()) < BUDGET:
    samples = planner.recommend(campaign.observations)
    for sample in samples:
        measurement = surface(sample)
        campaign.add_observation(sample, measurement)
        print(f'ITER : {iter_}\nSAMPLES : {samples}\t MEASUREMENT : {measurement}')
    iter_ += 1

Notice that, by default, `atlas` chooses which fidelity level to make the measurement for each call to `recommend` ($s$ changes between 1 and 0.5). However, an SDL researcher may want to further customize their multi-fidelity optimization campaign such that, for example, they can alternate between batches of low-fidelity measurements and high-fidelity measurements. 

`atlas` enables such flexible workflows by allowing the user to specify the fidelity level they wish for parameters to be recommended. To do this, one may set the `MultiFidelityPlanner`'s `current_ask_fidelity` attribute by calling the `set_ask_fidelity` method and specifying the desired level. `atlas` employs constrained acquisition function optimization to deliever the desired parameter recommendations. 

For our example, lets assume we simply want to alternate between low- and high-fidelity recommendations. Users can fully customize this aspect to suit thier specific SDL. 

In [ ]:
planner = MultiFidelityPlanner(
    goal='minimize', 
    init_design_strategy='random',
    num_init_design=5, 
    batch_size=1, 
    acquisition_optimizer_kind='pymoo',
    fidelity_params=2,
    fidelities=[0.5, 1.0],
)

planner.set_param_space(param_space)

campaign = Campaign()
campaign.set_param_space(param_space)

BUDGET = 20

iter_ = 0
while len(campaign.observations.get_values()) < BUDGET:
    
    # allow initial design to complete before alternating
    if len(campaign.observations.get_values()) >= 5:
        if iter_ % 2 == 0: # iterating between low and high fidelity samples
            planner.set_ask_fidelity(1.0) # target fidelity
        else:
            planner.set_ask_fidelity(0.5) # lower fidelity
                
    samples = planner.recommend(campaign.observations)
    for sample in samples:
        measurement = surface(sample)
        campaign.add_observation(sample, measurement)    
        print(f'ITER : {iter_}\nSAMPLES : {samples}\n MEASUREMENT : {measurement}')
    iter_ += 1

## Meta-/few-shot learning enhanced optimization

Users may incorporate data from historical optimization campaings via the `SourceTasks` module. These source tasks can be leveraged to accelerate the optimization rate on a novel campaign by using one of two meta-/few-shot learning planners: the Ranking-Weighted Gaussian Process Ensemble planner (`RGPEPlanner`) or the Deep Kernel Transfer planner (`DKTPlanner`).

In this illustrative example, lets optimize a simple trigonometric function with the help of simulated historical data from similar trigonometric functions. Let's define a target function, and use the `trig_factory` utility from `atlas` to produce 20 simulated historical optimmization campaigns.

In [ ]:
from atlas.utils.synthetic_data import trig_factory

def surface(x):
    return np.sin(8 * x)

# define the meta-training tasks
train_tasks = trig_factory(
    num_samples=20,
    scale_range=[[-8.5, -7.5], [7.5, 8.5]],
    shift_range=[-0.02, 0.02],
    amplitude_range=[0.2, 1.2],
)

valid_tasks = trig_factory(
    num_samples=5,
    scale_range=[[-8.5, -7.5], [7.5, 8.5]],
    shift_range=[-0.02, 0.02],
    amplitude_range=[0.2, 1.2],
)

The `train_tasks` and `valid_tasks` are lists of dictionaries, each of which represent one historical dataset/optimization trajectory. Each dictionary contains the keys `params` and `values` corresponding to `torch.Tensors` that contain the data points. 

We'll create a `ParameterSpace` for this problem, and define the meta-/few-shot learning planner. Here, we'll use the `RGPEPlanner`. We simply need to pass two additional arguments to the constructor, the `train_tasks` and `valid_tasks` we just generated.

In [ ]:
from atlas.planners.rgpe.planner import RGPEPlanner

param_space = ParameterSpace()
# add continuous parameter
param_0 = ParameterContinuous(name="param_0", low=0.0, high=1.0)
param_space.add(param_0)

planner = RGPEPlanner(
    goal="minimize",
    init_design_strategy='random',
    num_init_design=5,
    batch_size=1,
    acquisition_optimizer_kind='pymoo',
    # meta-learning stuff
    train_tasks=train_tasks,
    valid_tasks=valid_tasks,
#     cache_weights=False, 
#     hyperparams={},
)

planner.set_param_space(param_space)

# make the campaign
campaign = Campaign()
campaign.set_param_space(param_space)

BUDGET = 15

iter_ = 0
while len(campaign.observations.get_values()) < BUDGET:
    samples = planner.recommend(campaign.observations)
    for sample in samples:
        sample_arr = sample.to_array()
        measurement = surface(sample_arr)
        campaign.add_observation(sample_arr, measurement)
        print(f'ITER : {iter_}\nSAMPLES : {samples}\n MEASUREMENT : {measurement}')
        iter_+=1

The `RGPEPlanner` initial trains a separate GP model on each of the 20 source tasks, then proceeds with optimization on the novel problem. The contributions of each of the source GP models to the acquisition function are dynamically adjusted based on the observations of the novel problem. With the help of the information learned from the historical tasks, after the 5 initial design points are completed, we imediately converge to the novel problem's minimum value of $-1.0$ (`param_0`$\approx 0.58$). 

## Optimization over molecular parameter spaces

For optimization over molecular parameter spaces, we provide a specialized GP kernel function that is compatible with all planners based on the Tanimoto distance.

#### TODO: make an example here...

## Optimization with asynchronous execution

In many SDL applications, researchers have access to multiple robotic or computational workers and may parallelize measurements. When performing batched BO in a setting where there is variability in measurement times for each individual experiment, it is important to operate an SDL asynchronously, where a worker starts a new experiment immediatey after completion of the previous experiment.`atlas` implements aysnchronous optimization loosely based on the hallucination strategy first reported by Ginsbourger _et al._ This approach has been shown to be overall more efficient than waiting for an entire batch of experiments to complete before commencing the next batch.

For a simple demonstration of asynchronous execution using `atlas`, please try the [following example](https://github.com/aspuru-guzik-group/atlas/tree/main/tests/asynchronous_execution/example).